In [1]:
from trees import TreeMaker
from wrappers import GPTneoX_DenseWrapper, ActivationWrapper

import torch
import numpy as np
import umap
import plotly.graph_objects as go

/Users/collinszczepanski/Code/cris/LLM/chenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Install 'desc2025.xml' from MeSH website
tm = TreeMaker('desc2025.xml')

In [3]:
# View MeSH online browser to select a base tree number
key = 'E'

G, names = tm.tree_from_key(key, max_depth=1)

In [4]:
model = "EleutherAI/pythia-2.8b-deduped"
layer_num = 3

wrapper = ActivationWrapper(model)
layer = wrapper.make_layer_wrapper(layer_num, 'mlp')

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


In [5]:
wrapper.batch_logits(names, tokens = 'last').shape

torch.Size([374, 50278])

In [6]:
vocab_dict = wrapper.get_vocab()

In [8]:
#all_acts, all_toks = layer.batch_activations(names)
last_acts, last_toks = layer.batch_activations(names, tokens='last')

distances = torch.cdist(last_acts, last_acts, p=2)

In [9]:
umapper = umap.UMAP(n_components=3)
emb = umapper.fit_transform(last_acts.detach().numpy())
print(emb.shape)

x, y, z = emb[:, 0], emb[:, 1], emb[:, 2]
fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=4,
        opacity=0.8
    ),
    text=names,
    hoverinfo='text'        
)])

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    margin=dict(l=0, r=0, b=0, t=0)
)


/Users/collinszczepanski/Code/cris/LLM/chenv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


(374, 3)
